In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import scipy.io as scio
from sklearn.cluster import KMeans

In [2]:

Nx = 128
Ny = 128
m = Nx * Ny
# N number of samples 1000
# m number of points 40
# P number of output sensors (coordinates) 1600
# x_train is a tuple of u0(N,m) and output sensores, all coordinates xy_train_testing(P,2)
# y_train is target solutions (our s) u(N,P)

# tf.keras.utils.set_random_seed(seed)
fenics_data = scio.loadmat(
    "/scratch/bblv/qibang/repository/Adap_data_driven_possion/TrainingData/poisson_gauss_cov.mat"
)

x_grid = fenics_data["x_grid"].astype(np.float32)  # shape (Ny, Nx)
y_grid = fenics_data["y_grid"].astype(np.float32)
source_terms_raw = fenics_data["source_terms"].astype(
    np.float32)  # shape (N, Ny, Nx)
source_terms_raw = source_terms_raw.reshape(-1, Nx * Ny)
solutions_raw = fenics_data["solutions"].astype(
    np.float32)  # shape (N, Ny, Nx)
solutions_raw = solutions_raw.reshape(-1, Nx * Ny)
scaler_source = 0.5 * (np.max(source_terms_raw) - np.min(source_terms_raw))
scaler_solution = 0.5 * (np.max(solutions_raw) - np.min(solutions_raw))
solutions = solutions_raw / scaler_solution
source_terms = source_terms_raw / scaler_source
u0_train = source_terms[:-500]
u0_testing = source_terms[-500:]
s_train = solutions[:-500]
s_testing = solutions[-500:]

u0_testing_raw = source_terms_raw[-500:]
u0_train_raw = source_terms_raw[:-500]
s_testing_raw = solutions_raw[-500:]
s_train_raw = solutions_raw[:-500]

xy_train_testing = np.concatenate(
    [x_grid.reshape(-1, 1), y_grid.reshape(-1, 1)], axis=1
)



In [5]:
k = 100  # Number of clusters
kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=600,random_state=0)
y_kmeans = kmeans.fit_predict(u0_train)

/sw/external/python/anaconda3-2023.03_cuda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7f4429199870>
Traceback (most recent call last):
  File "/sw/external/python/anaconda3-2023.03_cuda/lib/python3.10/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/sw/external/python/anaconda3-2023.03_cuda/lib/python3.10/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/sw/external/python/anaconda3-2023.03_cuda/lib/python3.10/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_versio

In [6]:
np.savetxt(
        '/scratch/bblv/qibang/repository/Adap_data_driven_possion/TrainingData/poisson_gauss_cov_kmean100.txt',
        y_kmeans,
        fmt="%d",
        delimiter=",",
    )